# 🎾RandomForest с увеличенным числом деревьев и глубиной

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Функция для имитации реалистичных пропусков в теннисных данных
def create_realistic_tennis_missing_data(tennis_df, random_state=None):
    np.random.seed(random_state)
    df = tennis_df.copy()

    # 1. Рейтинг и очки чаще отсутствуют у малоизвестных игроков
    high_rank_mask = df['Rank_1'] > 100
    df.loc[high_rank_mask & (np.random.random(len(df)) < 0.3), 'Pts_1'] = np.nan

    high_rank_mask = df['Rank_2'] > 100
    df.loc[high_rank_mask & (np.random.random(len(df)) < 0.3), 'Pts_2'] = np.nan

    # 2. Коэффициенты могут отсутствовать для некоторых матчей
    df.loc[np.random.random(len(df)) < 0.1, ['Odd_1', 'Odd_2']] = np.nan

    # 3. Счет может быть неполным для незавершенных матчей
    # Имитируем отсутствие последних сетов
    def truncate_score(score, prob_missing=0.15):
        if pd.isna(score) or np.random.random() > prob_missing:
            return score

        sets = score.split()
        if len(sets) <= 1:
            return score

        # Сохраняем только первый сет с вероятностью 0.7, иначе первые два
        n_sets = 1 if np.random.random() < 0.7 else 2
        n_sets = min(n_sets, len(sets))

        return ' '.join(sets[:n_sets])

    df['Score'] = df['Score'].apply(lambda x: truncate_score(x))

    return df

# Функция для извлечения признаков из счета матча
def extract_score_features(score):
    if pd.isna(score):
        return [0] * 6
    
    sets = score.split()
    total_sets = len(sets)
    sets_won_1 = 0
    sets_won_2 = 0
    total_games_1 = 0
    total_games_2 = 0
    
    for set_score in sets:
        games = set_score.split('-')
        if len(games) == 2:
            games_1 = int(games[0])
            games_2 = int(games[1])
            total_games_1 += games_1
            total_games_2 += games_2
            if games_1 > games_2:
                sets_won_1 += 1
            else:
                sets_won_2 += 1
    
    return [total_sets, sets_won_1, sets_won_2, 
            total_games_1, total_games_2,
            total_games_1 + total_games_2]

# Функция для создания дополнительных признаков
def create_comparison_features(df):
    features = pd.DataFrame()
    
    # Рейтинговое преимущество (чем больше разница, тем сильнее преимущество)
    features['rank_advantage'] = (df['Rank_2'] - df['Rank_1']) / (df['Rank_1'] + df['Rank_2'])
    
    # Очковое преимущество
    features['points_advantage'] = (df['Pts_1'] - df['Pts_2']) / (df['Pts_1'] + df['Pts_2'])
    
    # Преимущество по коэффициентам
    features['odds_advantage'] = (df['Odd_2'] - df['Odd_1']) / (df['Odd_1'] + df['Odd_2'])
    
    # Бинарные признаки для прямого сравнения
    features['better_rank'] = (df['Rank_1'] < df['Rank_2']).astype(int)
    features['more_points'] = (df['Pts_1'] > df['Pts_2']).astype(int)
    features['better_odds'] = (df['Odd_1'] < df['Odd_2']).astype(int)
    
    # Составной показатель преимущества
    features['overall_advantage'] = (features['better_rank'] + 
                                   features['more_points'] + 
                                   features['better_odds']) / 3
    
    # Добавляем квадраты и взаимодействия признаков
    features['rank_points_interaction'] = features['rank_advantage'] * features['points_advantage']
    features['rank_odds_interaction'] = features['rank_advantage'] * features['odds_advantage']
    features['points_odds_interaction'] = features['points_advantage'] * features['odds_advantage']
    
    return features

# Обновленная функция подготовки данных
def prepare_data(df):
    df = df.copy()
    
    # Конвертируем дату в datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Заменяем отрицательные значения на NaN
    numeric_columns = ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']
    for col in numeric_columns:
        df.loc[df[col] < 0, col] = np.nan
    
    # Извлекаем признаки из счета
    score_features = df['Score'].apply(extract_score_features)
    score_features_df = pd.DataFrame(score_features.tolist(), 
                                   columns=['total_sets', 'sets_won_1', 'sets_won_2',
                                          'total_games_1', 'total_games_2', 'total_games'])
    
    # Создаем признак результата
    df['target'] = (df['Winner'] == df['Player_1']).astype(int)
    
    # Кодируем категориальные признаки
    le = LabelEncoder()
    categorical_columns = ['Series', 'Court', 'Surface', 'Round']
    for col in categorical_columns:
        df[col + '_encoded'] = le.fit_transform(df[col])
    
    # Добавляем новые признаки сравнения
    comparison_features = create_comparison_features(df)
    
    # Объединяем все признаки
    features = pd.concat([
        df[['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']],
        df[[col + '_encoded' for col in categorical_columns]],
        score_features_df,
        comparison_features
    ], axis=1)
    
    return features, df['target']

# Функция для обучения модели с настроенными параметрами
def train_model(df_train):
    X, y = prepare_data(df_train)
    
    # Заполняем пропущенные значения
    X = X.fillna(X.mean())
    
    # Создаем модель с увеличенным числом деревьев и настроенными параметрами
    model = RandomForestClassifier(
        n_estimators=200,           # Увеличиваем количество деревьев
        max_depth=10,               # Ограничиваем глубину для избежания переобучения
        min_samples_split=10,       # Минимальное количество образцов для разделения
        min_samples_leaf=5,         # Минимальное количество образцов в листе
        random_state=32,
        class_weight='balanced'     # Балансируем веса классов
    )
    model.fit(X, y)
    
    return model

# Функции get_predictions, create_results_table и display_results остаются без изменений
def get_predictions(model, df_test):
    X_test, _ = prepare_data(df_test)
    X_test = X_test.fillna(X_test.mean())
    probabilities = model.predict_proba(X_test)
    return probabilities

# Функция для создания таблицы результатов
def create_results_table(df_test, predictions):
    results = pd.DataFrame({
        'Player_1': df_test['Player_1'],
        'Player_1_Prediction': predictions[:, 1] * 100,
        'Player_2': df_test['Player_2'],
        'Player_2_Prediction': predictions[:, 0] * 100,
        'Winner': df_test['Winner'],
        'Rank_1': df_test['Rank_1'],
        'Rank_2': df_test['Rank_2'],
        'Pts_1': df_test['Pts_1'],
        'Pts_2': df_test['Pts_2'],
        'Odd_1': df_test['Odd_1'],
        'Odd_2': df_test['Odd_2'],
        'Score': df_test['Score']
    })
    
    # Определяем, кого модель прогнозирует победителем
    results['predicted_winner'] = np.where(predictions[:, 1] > predictions[:, 0], 
                                         df_test['Player_1'], 
                                         df_test['Player_2'])
    
    # Определяем, было ли предсказание верным
    results['correct_prediction'] = (results['predicted_winner'] == df_test['Winner'])
    
    # Добавляем уверенность в прогнозе - максимальная из двух вероятностей
    results['confidence'] = np.maximum(predictions[:, 0], predictions[:, 1]) * 100
    
    return results

# Функция для отображения результатов
def display_results(results):
    styled_results = results.style.format({
        'Player_1_Prediction': '{:.2f}%',
        'Player_2_Prediction': '{:.2f}%',
        'Rank_1': '{:.0f}',
        'Rank_2': '{:.0f}',
        'Pts_1': '{:.0f}',
        'Pts_2': '{:.0f}',
        'Odd_1': '{:.2f}',
        'Odd_2': '{:.2f}'
    }).apply(lambda x: ['background-color: #90EE90' if x['correct_prediction'] 
                       else 'background-color: #FFB6C6' for i in range(len(x))], axis=1)
    
    display(styled_results.hide(axis='columns', subset=['correct_prediction']))

# Функция для оценки и вывода метрик
def display_metrics(model, df_test, predictions, results):
    # Истинные значения (1 - если победил первый игрок, 0 - если второй)
    y_true = (df_test['Winner'] == df_test['Player_1']).astype(int)
    
    # Предсказанные вероятности победы первого игрока
    y_pred_proba = predictions[:, 1]
    
    # Предсказанные бинарные метки
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Расчет метрик
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred) * 100
    recall = recall_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred) * 100
    roc_auc = roc_auc_score(y_true, y_pred_proba) * 100
    entropy_loss = log_loss(y_true, y_pred_proba)
    
    # Матрица ошибок
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Выводим метрики
    print(f"\nМетрики качества модели:")
    print(f"Accuracy (Точность): {accuracy:.2f}%")
    print(f"Precision (Точность по положительному классу): {precision:.2f}%")
    print(f"Recall (Полнота): {recall:.2f}%")
    print(f"F1-score (F-мера): {f1:.2f}%")
    print(f"ROC AUC: {roc_auc:.2f}%")
    print(f"Log Loss: {entropy_loss:.4f}")
    
    # Анализ ошибок
    print("\nМатрица ошибок (Confusion Matrix):")
    print(f"True Negative: {tn} (Правильно предсказанные победы второго игрока)")
    print(f"False Positive: {fp} (Неправильно предсказанные победы первого игрока)")
    print(f"False Negative: {fn} (Неправильно предсказанные победы второго игрока)")
    print(f"True Positive: {tp} (Правильно предсказанные победы первого игрока)")
    
    # Расчет метрик по уверенности прогноза
    confidence_bins = [50, 60, 70, 80, 90, 100]
    print("\nТочность предсказаний по уровню уверенности:")
    
    for i in range(len(confidence_bins)-1):
        low = confidence_bins[i]
        high = confidence_bins[i+1]
        
        # Фильтруем предсказания по уровню уверенности
        confident_preds = results[(results['confidence'] >= low) & (results['confidence'] < high)]
        
        if len(confident_preds) > 0:
            bin_accuracy = (confident_preds['correct_prediction'].sum() / len(confident_preds)) * 100
            print(f"Уверенность {low}%-{high}%: {bin_accuracy:.2f}% точность ({len(confident_preds)} матчей)")
        else:
            print(f"Уверенность {low}%-{high}%: нет матчей")
    
    # Выводим важность признаков
    feature_names = (
        ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2'] +
        ['Series_encoded', 'Court_encoded', 'Surface_encoded', 'Round_encoded'] +
        ['total_sets', 'sets_won_1', 'sets_won_2', 'total_games_1', 'total_games_2', 'total_games'] +
        ['rank_advantage', 'points_advantage', 'odds_advantage', 'better_rank', 'more_points', 'better_odds', 'overall_advantage', 
         'rank_points_interaction', 'rank_odds_interaction', 'points_odds_interaction']
    )
    
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nВажность признаков:")
    print(feature_importance.head(10))

---

## 1. Обучение модели на большой выборке без пропущенных значений

In [2]:
df_train = pd.read_csv('atp_tennis.csv')

# Обучаем модель
model = train_model(df_train)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 64018


### Тестирование на выборке без пропущенных значений

In [3]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.00%,Dolgopolov O.,100.00%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,99.999896
1,Harrison R.,51.18%,Sela D.,48.82%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,51.184589
2,Anderson K.,51.18%,Edmund K.,48.82%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,51.179334
3,Carreno Busta P.,98.89%,Kubler J.,1.11%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,98.891027
4,Youzhny M.,0.04%,Cuevas P.,99.96%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.957148
5,Istomin D.,99.92%,Herbert P.H.,0.08%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.919718
6,Delbonis F.,0.40%,Muller G.,99.60%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.598085
7,Seppi A.,99.92%,Moutet C.,0.08%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,99.920741
8,Ferrer D.,41.54%,Rublev A.,58.46%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,58.461257
9,Ebden M.,98.98%,Isner J.,1.02%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,98.976848



Метрики качества модели:
Accuracy (Точность): 94.31%
Precision (Точность по положительному классу): 90.91%
Recall (Полнота): 98.36%
F1-score (F-мера): 94.49%
ROC AUC: 99.10%
Log Loss: 0.1269

Матрица ошибок (Confusion Matrix):
True Negative: 56 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 60 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 68.18% точность (22 матчей)
Уверенность 60%-70%: 100.00% точность (1 матчей)
Уверенность 70%-80%: нет матчей
Уверенность 80%-90%: нет матчей
Уверенность 90%-100%: 100.00% точность (100 матчей)

Важность признаков:
           Feature  Importance
11      sets_won_1    0.350146
12      sets_won_2    0.323941
14   total_games_2    0.111554
13   total_games_1    0.109784
4            Odd_1    0.018413
15     total_games    0.017441
5     

### Тестирование на выборке с пропущенными значениями

In [4]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,19.19%,Dolgopolov O.,80.81%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,80.805388
1,Harrison R.,51.18%,Sela D.,48.82%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,51.184589
2,Anderson K.,51.18%,Edmund K.,48.82%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,51.179334
3,Carreno Busta P.,98.89%,Kubler J.,1.11%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,98.891027
4,Youzhny M.,0.04%,Cuevas P.,99.96%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.957148
5,Istomin D.,99.92%,Herbert P.H.,0.08%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.919718
6,Delbonis F.,0.40%,Muller G.,99.60%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.598085
7,Seppi A.,35.68%,Moutet C.,64.32%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,64.317247
8,Ferrer D.,41.54%,Rublev A.,58.46%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,58.461257
9,Ebden M.,81.38%,Isner J.,18.62%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Ebden M.,81.383962



Метрики качества модели:
Accuracy (Точность): 91.87%
Precision (Точность по положительному классу): 89.23%
Recall (Полнота): 95.08%
F1-score (F-мера): 92.06%
ROC AUC: 98.52%
Log Loss: 0.1427

Матрица ошибок (Confusion Matrix):
True Negative: 55 (Правильно предсказанные победы второго игрока)
False Positive: 7 (Неправильно предсказанные победы первого игрока)
False Negative: 3 (Неправильно предсказанные победы второго игрока)
True Positive: 58 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 59.09% точность (22 матчей)
Уверенность 60%-70%: 50.00% точность (2 матчей)
Уверенность 70%-80%: нет матчей
Уверенность 80%-90%: 100.00% точность (3 матчей)
Уверенность 90%-100%: 100.00% точность (96 матчей)

Важность признаков:
           Feature  Importance
11      sets_won_1    0.350146
12      sets_won_2    0.323941
14   total_games_2    0.111554
13   total_games_1    0.109784
4            Odd_1    0.018413
15     total_games    

---

## 2. Обучение модели на большой выборке с пропущенными значениями

In [5]:
df_train = pd.read_csv('atp_tennis.csv')

# Имитируем пропущенные значения в обучающей выборке
df_train_missing = create_realistic_tennis_missing_data(df_train, random_state=32)

# Обучаем модель
model = train_model(df_train_missing)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 64018


### Тестирование на выборке без пропущенных значений

In [6]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.01%,Dolgopolov O.,99.99%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,99.993353
1,Harrison R.,50.52%,Sela D.,49.48%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,50.518192
2,Anderson K.,49.54%,Edmund K.,50.46%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Edmund K.,50.464891
3,Carreno Busta P.,99.39%,Kubler J.,0.61%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.385769
4,Youzhny M.,0.05%,Cuevas P.,99.95%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.951633
5,Istomin D.,99.48%,Herbert P.H.,0.52%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.483695
6,Delbonis F.,0.02%,Muller G.,99.98%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.982095
7,Seppi A.,99.65%,Moutet C.,0.35%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,99.647625
8,Ferrer D.,40.38%,Rublev A.,59.62%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,59.620566
9,Ebden M.,99.39%,Isner J.,0.61%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,99.385077



Метрики качества модели:
Accuracy (Точность): 94.31%
Precision (Точность по положительному классу): 90.91%
Recall (Полнота): 98.36%
F1-score (F-мера): 94.49%
ROC AUC: 99.13%
Log Loss: 0.1282

Матрица ошибок (Confusion Matrix):
True Negative: 56 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 60 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 69.57% точность (23 матчей)
Уверенность 60%-70%: нет матчей
Уверенность 70%-80%: нет матчей
Уверенность 80%-90%: нет матчей
Уверенность 90%-100%: 100.00% точность (100 матчей)

Важность признаков:
              Feature  Importance
11         sets_won_1    0.334055
12         sets_won_2    0.311537
14      total_games_2    0.104610
13      total_games_1    0.091750
10         total_sets    0.033904
15        total_games    0.031095
4 

### Тестирование на выборке с пропущенными значениями

In [7]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,6.18%,Dolgopolov O.,93.82%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,93.820631
1,Harrison R.,50.52%,Sela D.,49.48%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,50.518192
2,Anderson K.,49.54%,Edmund K.,50.46%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Edmund K.,50.464891
3,Carreno Busta P.,99.39%,Kubler J.,0.61%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.385769
4,Youzhny M.,0.05%,Cuevas P.,99.95%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.951633
5,Istomin D.,99.48%,Herbert P.H.,0.52%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.483695
6,Delbonis F.,0.02%,Muller G.,99.98%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.982095
7,Seppi A.,42.04%,Moutet C.,57.96%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,57.963551
8,Ferrer D.,40.38%,Rublev A.,59.62%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,59.620566
9,Ebden M.,57.02%,Isner J.,42.98%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Ebden M.,57.019852



Метрики качества модели:
Accuracy (Точность): 92.68%
Precision (Точность по положительному классу): 90.62%
Recall (Полнота): 95.08%
F1-score (F-мера): 92.80%
ROC AUC: 98.65%
Log Loss: 0.1438

Матрица ошибок (Confusion Matrix):
True Negative: 56 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 3 (Неправильно предсказанные победы второго игрока)
True Positive: 58 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 64.00% точность (25 матчей)
Уверенность 60%-70%: нет матчей
Уверенность 70%-80%: нет матчей
Уверенность 80%-90%: 100.00% точность (1 матчей)
Уверенность 90%-100%: 100.00% точность (97 матчей)

Важность признаков:
              Feature  Importance
11         sets_won_1    0.334055
12         sets_won_2    0.311537
14      total_games_2    0.104610
13      total_games_1    0.091750
10         total_sets    0.033904
15        total_game

---

## 3. Обучение модели на маленькой выборке без пропущенных значений

In [8]:
df_train = pd.read_csv('atp_tennis_47302_xxs.csv')

# Обучаем модель
model = train_model(df_train)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 50


### Тестирование на выборке без пропущенных значений

In [9]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,5.01%,Dolgopolov O.,94.99%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,94.988956
1,Harrison R.,57.79%,Sela D.,42.21%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,57.794713
2,Anderson K.,54.93%,Edmund K.,45.07%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,54.934516
3,Carreno Busta P.,81.37%,Kubler J.,18.63%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,81.368091
4,Youzhny M.,16.12%,Cuevas P.,83.88%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,83.879272
5,Istomin D.,86.32%,Herbert P.H.,13.68%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,86.321257
6,Delbonis F.,10.82%,Muller G.,89.18%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,89.184487
7,Seppi A.,80.28%,Moutet C.,19.72%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,80.279757
8,Ferrer D.,51.51%,Rublev A.,48.49%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,51.508118
9,Ebden M.,44.09%,Isner J.,55.91%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Isner J.,55.913938



Метрики качества модели:
Accuracy (Точность): 91.87%
Precision (Точность по положительному классу): 89.23%
Recall (Полнота): 95.08%
F1-score (F-мера): 92.06%
ROC AUC: 97.78%
Log Loss: 0.3022

Матрица ошибок (Confusion Matrix):
True Negative: 55 (Правильно предсказанные победы второго игрока)
False Positive: 7 (Неправильно предсказанные победы первого игрока)
False Negative: 3 (Неправильно предсказанные победы второго игрока)
True Positive: 58 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 55.56% точность (18 матчей)
Уверенность 60%-70%: 96.00% точность (25 матчей)
Уверенность 70%-80%: 90.00% точность (10 матчей)
Уверенность 80%-90%: 100.00% точность (61 матчей)
Уверенность 90%-100%: 100.00% точность (9 матчей)

Важность признаков:
             Feature  Importance
11        sets_won_1    0.252662
12        sets_won_2    0.192996
14     total_games_2    0.103786
4              Odd_1    0.092801
18    odds_advantage    0

### Тестирование на выборке с пропущенными значениями

In [10]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,27.91%,Dolgopolov O.,72.09%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,72.088956
1,Harrison R.,57.79%,Sela D.,42.21%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,57.794713
2,Anderson K.,54.93%,Edmund K.,45.07%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,54.934516
3,Carreno Busta P.,81.37%,Kubler J.,18.63%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,81.368091
4,Youzhny M.,16.12%,Cuevas P.,83.88%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,83.879272
5,Istomin D.,86.32%,Herbert P.H.,13.68%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,86.321257
6,Delbonis F.,10.82%,Muller G.,89.18%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,89.184487
7,Seppi A.,63.23%,Moutet C.,36.77%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Seppi A.,63.227177
8,Ferrer D.,51.51%,Rublev A.,48.49%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,51.508118
9,Ebden M.,31.48%,Isner J.,68.52%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Isner J.,68.524165



Метрики качества модели:
Accuracy (Точность): 91.06%
Precision (Точность по положительному классу): 87.88%
Recall (Полнота): 95.08%
F1-score (F-мера): 91.34%
ROC AUC: 96.77%
Log Loss: 0.3328

Матрица ошибок (Confusion Matrix):
True Negative: 54 (Правильно предсказанные победы второго игрока)
False Positive: 8 (Неправильно предсказанные победы первого игрока)
False Negative: 3 (Неправильно предсказанные победы второго игрока)
True Positive: 58 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 63.16% точность (19 матчей)
Уверенность 60%-70%: 89.66% точность (29 матчей)
Уверенность 70%-80%: 93.33% точность (15 матчей)
Уверенность 80%-90%: 100.00% точность (55 матчей)
Уверенность 90%-100%: 100.00% точность (5 матчей)

Важность признаков:
             Feature  Importance
11        sets_won_1    0.252662
12        sets_won_2    0.192996
14     total_games_2    0.103786
4              Odd_1    0.092801
18    odds_advantage    0

---

## 4. Обучение модели на маленькой выборке с пропущенными значениями

In [11]:
df_train = pd.read_csv('atp_tennis_47302_xxs.csv')

# Имитируем пропущенные значения в обучающей выборке
df_train_missing = create_realistic_tennis_missing_data(df_train, random_state=32)

# Обучаем модель
model = train_model(df_train_missing)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 50


### Тестирование на выборке без пропущенных значений

In [12]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,3.78%,Dolgopolov O.,96.22%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,96.218821
1,Harrison R.,67.75%,Sela D.,32.25%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,67.751877
2,Anderson K.,63.71%,Edmund K.,36.29%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,63.710777
3,Carreno Busta P.,71.36%,Kubler J.,28.64%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,71.358975
4,Youzhny M.,14.90%,Cuevas P.,85.10%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,85.103127
5,Istomin D.,68.43%,Herbert P.H.,31.57%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,68.432967
6,Delbonis F.,11.34%,Muller G.,88.66%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,88.656789
7,Seppi A.,68.38%,Moutet C.,31.62%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,68.382118
8,Ferrer D.,58.67%,Rublev A.,41.33%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,58.673824
9,Ebden M.,30.66%,Isner J.,69.34%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Isner J.,69.338080



Метрики качества модели:
Accuracy (Точность): 90.24%
Precision (Точность по положительному классу): 88.89%
Recall (Полнота): 91.80%
F1-score (F-мера): 90.32%
ROC AUC: 95.77%
Log Loss: 0.3635

Матрица ошибок (Confusion Matrix):
True Negative: 55 (Правильно предсказанные победы второго игрока)
False Positive: 7 (Неправильно предсказанные победы первого игрока)
False Negative: 5 (Неправильно предсказанные победы второго игрока)
True Positive: 56 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 80.00% точность (20 матчей)
Уверенность 60%-70%: 77.42% точность (31 матчей)
Уверенность 70%-80%: 94.74% точность (19 матчей)
Уверенность 80%-90%: 100.00% точность (45 матчей)
Уверенность 90%-100%: 100.00% точность (8 матчей)

Важность признаков:
             Feature  Importance
11        sets_won_1    0.233223
12        sets_won_2    0.183763
18    odds_advantage    0.076872
16    rank_advantage    0.070265
13     total_games_1    0

### Тестирование на выборке с пропущенными значениями

In [13]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,9.75%,Dolgopolov O.,90.25%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,90.252028
1,Harrison R.,67.75%,Sela D.,32.25%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,67.751877
2,Anderson K.,63.71%,Edmund K.,36.29%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,63.710777
3,Carreno Busta P.,71.36%,Kubler J.,28.64%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,71.358975
4,Youzhny M.,14.90%,Cuevas P.,85.10%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,85.103127
5,Istomin D.,68.43%,Herbert P.H.,31.57%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,68.432967
6,Delbonis F.,11.34%,Muller G.,88.66%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,88.656789
7,Seppi A.,48.22%,Moutet C.,51.78%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,51.784211
8,Ferrer D.,58.67%,Rublev A.,41.33%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,58.673824
9,Ebden M.,30.37%,Isner J.,69.63%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Isner J.,69.633648



Метрики качества модели:
Accuracy (Точность): 89.43%
Precision (Точность по положительному классу): 88.71%
Recall (Полнота): 90.16%
F1-score (F-мера): 89.43%
ROC AUC: 95.37%
Log Loss: 0.3708

Матрица ошибок (Confusion Matrix):
True Negative: 55 (Правильно предсказанные победы второго игрока)
False Positive: 7 (Неправильно предсказанные победы первого игрока)
False Negative: 6 (Неправильно предсказанные победы второго игрока)
True Positive: 55 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 76.19% точность (21 матчей)
Уверенность 60%-70%: 76.67% точность (30 матчей)
Уверенность 70%-80%: 95.45% точность (22 матчей)
Уверенность 80%-90%: 100.00% точность (45 матчей)
Уверенность 90%-100%: 100.00% точность (5 матчей)

Важность признаков:
             Feature  Importance
11        sets_won_1    0.233223
12        sets_won_2    0.183763
18    odds_advantage    0.076872
16    rank_advantage    0.070265
13     total_games_1    0